# 09wk-1: 추천시스템

최규빈  
2024-04-29

<a href="https://colab.research.google.com/github/guebin/DL2024/blob/main/posts/08wk-1-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [1]:
#{{<video https://youtu.be/playlist?list=PLQqh36zP38-xfh-AQQI0B_GONOjgj9DCi&si=uWImDc1bYBoNqCB_ >}}

# 2. Imports

In [2]:
import torch
import numpy as np 
import pandas as pd
import fastai.collab

# 4. 나는 SOLO

ref: <https://namu.wiki/w/나는%20SOLO>

## A. Data

`-` Data

In [3]:
df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/STML2022/main/posts/V.%20RecSys/2022-12-21-rcmdsolo.csv',index_col=0)
df_view 

`-` 데이터를 이해할 때 필요한 가정들 – 제가 마음대로 설정했어요..

-   궁합이 잘맞으면 5점, 잘 안맞으면 0점 이다.
-   ((옥순,영자,정숙),(영식,영철,영호))은 MBTI가 I로 시작하고
    ((영숙,순자,현숙),(광수,상철,영수))는 MBTI가 E로 시작한다고
    설정하자.
-   (옥순,영자,정숙)은 (영식,영철,영호)와 성격이 잘 맞고
    (영숙,순자,현숙)은 (광수,상철,영수)와 성격이 잘맞음.
-   정숙은 대체적으로 모든 사람들이랑 궁합이 잘 맞는 편인것 같다.
-   현숙은 전체적으로 모든 사람들이랑 궁합이 잘 안맞는 편인것 같다.
    (눈이 높아보인다)

## B. Fit / Predict

`-` 목표: NaN을 추정

`-` 수동추론:

-   (옥순,영호)이 만난다면? $\to$ 둘다 I성향이니까 잘 맞지 않을까? $\to$
    4.0 정도?
-   (정숙,영식)조합은? $\to$ 둘다 I성향이니까 잘 맞지 않을까? + 정숙은
    다 잘맞던데..? $\to$ 4.8 정도?
-   (현숙,영식)조합은? $\to$ 현숙은 E성향인데 영식은 I성향이므로 잘
    안맞을 것임 + 현숙은 원래 좀 눈이 높음 $\to$ 0.25 정도?

`-` 좀 더 체계적인 추론 전략: 사람들이 가지고 있는 성향들을 두 개의
숫자로 표현하자.

-   옥순의 성향 = (I성향,E성향) = (1.9, 0.0)
-   영식의 성향 = (I성향,E성향) = (2.0, 0.1)
-   현숙의 성향 = (I성향,E성향) = (0.0, 1.5)

> 사람에 따라 E,I 둘다 높게 설정할수도있고, 둘다 낮게 설정할 수도 있다.

**(1)** 옥순과 영식의 궁합 $\approx$ 옥순의I성향$\times$영식의I성향 $+$
옥순의E성향$\times$영식의E성향 // 적합

In [4]:
옥순성향 = torch.tensor([1.9,0.0]).reshape(2,1) 
영식성향 = torch.tensor([2.0,0.1]).reshape(2,1) 
#(옥순성향*영식성향).sum().item() # 옥순과 영식의 궁합: a ∘ b 로 내적구함 
(옥순성향.T @ 영식성향).item() # 옥순과 영식의 궁합: a.T @ b 로 내적구함 

**(2)** 현숙과 영식의 궁합 $\approx$ 현숙의I성향$\times$영식의I성향 $+$
현숙의E성향$\times$영식의E성향 // 예측

In [5]:
현숙성향 = torch.tensor([0.0,1.5]).reshape(2,1)
#(현숙성향*영식성향).sum().item() # 현숙과 영식의 궁합: a ∘ b 로 내적구함 
(현숙성향.T @ 영식성향).item() # 현숙과 영식의 궁합: a.T @ b 로 내적구함 

-   그럴싸함.

`-` 전체 사용자의 설정값

In [101]:
옥순성향 = a1 = torch.tensor([1.9,0.0]).reshape(2,1)
영자성향 = a2 = torch.tensor([2.0,0.1]).reshape(2,1)
정숙성향 = a3 = torch.tensor([2.5,1.0]).reshape(2,1)
영숙성향 = a4 = torch.tensor([0.1,1.9]).reshape(2,1)
순자성향 = a5 = torch.tensor([0.2,2.1]).reshape(2,1)
현숙성향 = a6 = torch.tensor([0.0,1.5]).reshape(2,1)
A = torch.concat([a1,a2,a3,a4,a5,a6],axis=1)
A # 각 column은 여성출연자들의 성향을 의미함 

In [102]:
영식성향 = b1 = torch.tensor([2.0,0.1]).reshape(2,1)
영철성향 = b2 = torch.tensor([1.9,0.2]).reshape(2,1)
영호성향 = b3 = torch.tensor([1.8,0.3]).reshape(2,1)
광수성향 = b4 = torch.tensor([0.3,2.1]).reshape(2,1)
상철성향 = b5 = torch.tensor([0.2,2.0]).reshape(2,1)
영수성향 = b6 = torch.tensor([0.1,1.9]).reshape(2,1)
B = torch.concat([b1,b2,b3,b4,b5,b6],axis=1)
B # 각 column은 남성출연자의 성향을 의미함

`-` 아래의 행렬곱 관찰

In [8]:
A.T@B 

—저거 따져보자—

${\bf A}_{2 \times 6} = \begin{bmatrix} {\boldsymbol a}_{\text{옥순}} & {\boldsymbol a}_{\text{영자}} & {\boldsymbol a}_{\text{정숙}} & {\boldsymbol a}_{\text{영숙}} & {\boldsymbol a}_{\text{순자}} & {\boldsymbol a}_{\text{현숙}} \end{bmatrix} = \begin{bmatrix} 1.9 & 2.0 & 2.5 & 0.1 & 0.2 & 0 \\ 0 & 0.1 & 1.0 & 1.9 & 2.1 & 1.5 \end{bmatrix}$

${\bf B}_{2 \times 6} = \begin{bmatrix} {\boldsymbol b}_{\text{영식}} & {\boldsymbol b}_{\text{영철}} & {\boldsymbol b}_{\text{영호}} & {\boldsymbol b}_{\text{광수}} & {\boldsymbol b}_{\text{상철}} & {\boldsymbol b}_{\text{영수}} \end{bmatrix} = \begin{bmatrix} 2.0 & 1.0 & 1.8 & 0.3 & 0.2 & 0.1 \\ 0.1 & 0.2 & 0.3 & 2.1 & 2.0 & 1.9 \end{bmatrix}$

$\begin{align*}{\bf A}^\top @ {\bf B} & = \begin{bmatrix} 1.9 & 0.0 \\ 2.0 & 0.1 \\ 2.5 & 1.0 \\ 0.1 & 1.9 \\ 0.2 & 2.1 \\ 0.0 & 1.5 \\ \end{bmatrix} \begin{bmatrix} 2.0 & 1.0 & 1.8 & 0.3 & 0.2 & 0.1 \\ 0.1 & 0.2 & 0.3 & 2.1 & 2.0 & 1.9 \end{bmatrix} \\ \\ & = \begin{bmatrix} 3.8 & 3.61 & 3.42 & 0.57 & 0.38 & 0.19 \\ 4.01 & 3.82 & 3.63 & 0.81 & 0.6 & 0.39 \\ 5.1 & 4.95 & 4.8 & 2.85 & 2.5 & 2.15 \\ 0.39 & 0.57 & 0.75 & 4.02 & 3.82 & 3.62 \\ 0.61 & 0.8 & 0.99 & 4.47 & 4.24 & 4.01 \\ 0.15 & 0.3 & 0.45 & 3.15 & 3 & 2.85 \\ \end{bmatrix} \\ \\ & =\begin{bmatrix} {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\ {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\ {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\ {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\ {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\ {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\ \end{bmatrix}\end{align*}$

`-` 궁합매트릭스: ${\bf A}^\top @ {\bf B}$를 계산하면 (6,6) 인 행렬이
나올텐데 이 행렬의 $(i,j)$의 원소는 $i$-th 여성출연자와 $j$-th
남성출연자가 얼마나 잘 맞는지를 나타내는 숫자가 된다. (숫자가 높을수록
잘 맞음) 이 수업에서는 이 매트릭스를 “궁합매트릭스” 라고 정의하자.

`-` 주어진 자료와 우리가 임의로 만든 궁합매트릭스를 비교해보자.

In [9]:
A.T @ B, torch.tensor(np.array(df_view)).float()

`-` 우리의 전략

1.  ${\bf A}^\top @ {\bf B}$의 값과 `df_view` 의 값이 `nan`을 제외한
    곳에서 거의 비슷하게 되도록 ${\bf A}$, ${\bf B}$를 잘 때려맞추면
    되는것 아니야?

2.  1을 만족하는 ${\bf A}$, ${\bf A}$를 찾았으면 그 숫자들을 이용하여
    `df_view`의 `nan` 을 추정한다.

`-` 따라서 모형은 아래와 같이 볼 수 있다.

$$\textbf{df_view} \approx {\bf A}^\top @ {\bf B}$$

`-` 아래의 정보를 참고하여 위의 수식을 다시 정리하면..

In [15]:
df_view

In [29]:
display(pd.DataFrame(A.T.tolist()).set_axis(['옥순','영자','정숙','영숙','순자','현숙']).set_axis(['I성향','E성향'],axis=1))
display(pd.DataFrame(B.T.tolist()).set_axis(['영식','영철','영호','광수','상철','영수']).set_axis(['I성향','E성향'],axis=1))

—이런식으로?—

|  X1  |  X2  | ${\boldsymbol a}_{X1}^T$ | ${\boldsymbol b}_{X2}^T$ | yhat |  y  |
|:----:|:----:|:------------------------:|:------------------------:|:----:|:---:|
| 옥순 | 영식 |       \[1.9, 0.0\]       |       \[2.0, 0.1\]       | 3.8  | 3.9 |
| 옥순 | 영철 |       \[1.9, 0.0\]       |       \[1.9, 0.2\]       | 3.6  | 4.1 |
| 옥순 | 광수 |       \[1.9, 0.0\]       |       \[0.3, 2.1\]       | 0.57 | 0.5 |
| 옥순 | 상철 |       \[1.9, 0.0\]       |       \[0.2, 2.0\]       | 0.38 | 0.3 |
|  …   |  …   |            …             |            …             |  …   |  …  |
| 현숙 | 영철 |       \[0.0, 1.5\]       |       \[1.9, 0.2\]       | 0.3  | 0.2 |
| 현숙 | 영호 |       \[0.0, 1.5\]       |       \[1.8, 0.3\]       | 0.45 | 0.2 |
| 현숙 | 상철 |       \[0.0, 1.5\]       |       \[0.2, 2.0\]       | 3.0  | 0.2 |
| 현숙 | 영수 |       \[0.0, 1.5\]       |       \[0.1, 1.9\]       | 2.85 | 0.2 |

`-` 이렇게 정리하면 `yhat` $\approx$ `y` 를 만족하도록 하면 된다.

-   `yhat`은 어떻게 구하지? `(A*B).sum()`?
-   그럼 `X1의[I,E]`,`X2의[I,E]`는 어떻게 구하지?? 생각해보니까
    데이터에서 주어진건 아니잖아

`-` `X1의[I,E]`,`X2의[I,E]` 를 어떻게 만들지?

-   그전엔 어떻게 했지? `X1`을 보고 적당히 특징을 상상하고 `X1`의
    `[I,E]`를 때려넣음, `X2`를 보고 적당히 특징을 상상하고 `X2`의
    `[I,E]`를 때려넣음.
-   자동화하려면? `X1` $\to$ \[`X1_I`, `X1_E`\] 인 함수를 만들고 `X2`
    $\to$ \[`X2_I`,`X2_E`\] 인 함수를 만들자.

`-` 앞으로 할일1: `X1` $\to$ \[`X1_I`, `X1_E`\] 인 함수를 만들고 `X2`
$\to$ \[`X2_I`,`X2_E`\] 인 함수를 만들자. 즉

-   옥순 $\to$ 1.9, 0.0
-   영자 $\to$ 2.0, 0.1 …
-   현숙 $\to$ 0.0, 1.5

로 가는 함수를 하나 구현하고

-   영식 $\to$ 2.0, 0.1
-   영철 $\to$ 1.9, 0.2 …
-   영수 $\to$ 0.1, 1.9

를 와 같은 함수도 하나 구현한다.

`-` 앞으로 할일2: 우리가 익숙한 셋팅

1.  `X1_I`, `X1_E`, `X2_I`, `X2_E` $\to$ `yhat` 를 수행
2.  `y` $\approx$ `yhat` 인지 체크: `loss = loss_fn(yhat,y)`
3.  `loss.backward()`
4.  더 나은 `X1_I`, `X1_E`, `X2_I`, `X2_E` 가 생기도록 update

## C. X1 $\to$ Z1, X2 $\to$ Z2 의 구현 – 실패

`-` dataframe의 변형

In [91]:
df = df_view.stack().reset_index().set_axis(['X1','X2','y'],axis=1)
df[:5]

`-` 이름을 숫자화

In [92]:
w = {'옥순':0, '영자':1, '정숙':2, '영숙':3, '순자':4, '현숙':5}
m = {'영식':0, '영철':1, '영호':2, '광수':3, '상철':4, '영수':5}

In [93]:
df['X1'] = df['X1'].map(w)
df['X2'] = df['X2'].map(m)
df[:5]

`-` X1 -\> Z1, X2 -\> Z2 로가는 변환구현

In [100]:
X1 = torch.tensor(df['X1']).reshape(-1,1).float()
X2 = torch.tensor(df['X2']).reshape(-1,1).float()
l1 = torch.nn.Linear(1,2)
l2 = torch.nn.Linear(1,2)
Zdf = pd.DataFrame(torch.concat([l1(X1),l2(X2)],axis=1)).round(2)
Zdf.set_axis[Z
pd.concat([df,Zdf],axis=1)[::2] 

성공했나?